In [ ]:
import glob #filenames and pathnames utility
import os   #operating sytem utility
import warnings

import flowgatenist as flow
#from flowgatenist import gaussian_mixture as nist_gmm
import flowgatenist.batch_process as batch_p

from Bio.Seq import Seq

import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates
import matplotlib.patches as patches
#from matplotlib.backends.backend_pdf import PdfPages

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
#from scipy import special
#from scipy import misc

import cmdstanpy
import gsf_ims_fitness.stan_utility as stan_utility
import pickle

import seaborn as sns
sns.set()

%load_ext autoreload
%autoreload 2

%matplotlib inline

# set global default style:
sns.set_style("white")
sns.set_style("ticks", {'xtick.direction':'in', 'xtick.top':True, 'ytick.direction':'in', 'ytick.right':True, })
#sns.set_style({"axes.labelsize": 20, "xtick.labelsize" : 16, "ytick.labelsize" : 16})

plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.edgecolor'] = 'k'

plt.rcParams['xtick.major.pad'] = 8

Indicate the directory where the notebook is saved:

In [ ]:
ms=4*1.3

In [ ]:
notebook_dir = os.getcwd()
notebook_dir

In [ ]:
os.chdir(notebook_dir)

file_name = 'Ligafy_cytom_data_all.pkl'
full_summary = pickle.load(open(file_name, 'rb'))

pickle_file = 'Ligafy_cytom_Hill_fit_results_mean.pkl'
variant_table = pickle.load(open(pickle_file, 'rb'))


pickle_file = 'Ligafy_cytom_Hill_fits_mean.pkl'
model_pickle_file = 'Ligafy_cytom_Hill_fits_stan_model.pkl'
pickled_model = pickle.load(open(model_pickle_file, 'rb'))
cytom_Hill_fits = pickle.load(open(pickle_file, 'rb'))

In [ ]:
cytom_Hill_fits.keys()

In [ ]:
sf = cytom_Hill_fits[('pLigify-LcLacI', 'D-arabitol')][3]

In [ ]:
x = sf.stan_variable('ec50')
np.mean(x), np.std(x)

In [ ]:
x = cytom_Hill_fits[('pLigify-LcLacI', 'D-arabitol')][0].values
max(x)

In [ ]:
variant_table

In [ ]:
def hill_funct(x, low, high, mid, n):
    return low + (high-low)*( x**n )/( mid**n + x**n )

In [ ]:
def add_break(ax, x_0=0.11, w=0.04, d_x=0.01, d_y=0.025, zorder=2000):
    rect = [(x_0, -0.025), w, 1.05]
    ax.add_patch(patches.Rectangle(*rect, transform=ax.transAxes, clip_on=False, zorder=zorder, color='w'));
    zorder += 1
    for y_0 in [0, 1]:
        ax.plot([x_0-d_x, x_0+d_x], [y_0-d_y, y_0+d_y], transform=ax.transAxes, clip_on=False, zorder=zorder, color='k')
        ax.plot([x_0-d_x+w, x_0+d_x+w], [y_0-d_y, y_0+d_y], transform=ax.transAxes, clip_on=False, zorder=zorder, color='k')

In [ ]:
np.unique(full_summary.variant)

In [ ]:
len(np.unique(full_summary.variant))

In [ ]:
np.unique(full_summary.inducerId)

In [ ]:
variant_ligand_pairs = [['pLigify-IemR', 'isoeugenol'], 
                        ['pLigify-LcLacI', 'D-ribose'], 
                        ['pLigify-SorR', 'L-sorbose'], 
                        ['pLigify-VprR', '4-vinylphenol']]

In [ ]:
df = full_summary
df = df[df.variant=='pAN-1201']

non_fluorescent_mean = df['mean'].mean()
non_fluorescent_mean_err = df['mean'].std()

ok_to_fail_fit = []
non_fluorescent_mean, non_fluorescent_mean_err

In [ ]:
full_summary['date_plate_row'] = [f'{a}_{b}' for a, b in zip(full_summary['date_plate'], full_summary['row'])]

In [ ]:
# Plot data with fits to check fit quality
os.chdir(notebook_dir)
if not os.path.exists('dose_response_plots'):
    os.makedirs('dose_response_plots')
os.chdir('dose_response_plots')
plt.rcParams["figure.figsize"] = [6, 4]

color = sns.color_palette()[0]

for pair, linthresh in zip(variant_ligand_pairs, [0.001, 10, 0.1, 0.1]):
    var = pair[0]
    lig = pair[1]
    df_0 = full_summary
    df_0 = df_0[~df_0.is_bad_rep]
    df_0 = df_0[df_0.variant==var]

    df_0 = df_0[(df_0.inducerId==lig)|(df_0.inducerId=='none')]

    fig, ax = plt.subplots(1, 1)
    #fig.suptitle(f'{var}, {lig}', size=20)
    
    x_0 = df_0.inducerConcentration
    x_min = min(x_0[x_0>0])
    x_max = max(x_0)
    x_fit = [0] + list(np.logspace(np.log10(x_min/10), np.log10(x_max*2), 40))

    variant_row = variant_table[(variant_table.variant==var)&(variant_table.ligand==lig)].iloc[0]
    hill_params = [variant_row[p] for p in ['log_g0', 'log_ginf', 'log_ec50', 'n']]
    hill_params = [10**p for p in hill_params[:3]] + hill_params[3:]
    y_fit = hill_funct(x_fit, *hill_params)
    
    for date_plate, df in df_0.groupby('date_plate_row'):
        if len(df)>0:
            x = df.inducerConcentration
            y = df['mean'] - non_fluorescent_mean
            yerr = df['mean_err']
            ax.errorbar(x, y/1000, yerr/1000, fmt='o', ms=ms, color=color)

    ax.plot(x_fit, y_fit/1000, color=color)

    ax.set_xscale('symlog', linthresh=linthresh);
    ax.set_xlabel(f'[{lig}] (µmol/L)');
    ax.set_ylabel(f'Output (kMEF)');
    add_break(ax)
    
    fig_file = f'{var}_{lig}.dose_response.svg'
    fig.savefig(fig_file, bbox_inches='tight')
    
os.chdir(notebook_dir)

In [ ]:
plot_list = [('pLigify-IemR', 'isoeugenol'),
 ('pLigify-LcLacI', 'D-ribose'),
 ('pLigify-SorR', 'L-sorbose'),
 ('pLigify-VprR', '4-vinylphenol'),
 ('pLigify-IemR', 'eugenol'),
 ('pLigify-IemR', '4-ethylguaiacol'),
 ('pLigify-IemR', '4-vinylphenol'),
 ('pLigify-VprR', '4-ethylphenol'),
 ('pLigify-VprR', '4-ethylguaiacol'),
 ('pLigify-VprR', 'isoeugenol'),
 ('pLigify-LcLacI', 'L-ribose'),
 ('pLigify-LcLacI', 'D-arabitol'),
 ('pLigify-LcLacI', 'L-arabitol'),
 ('pLigify-SorR', 'D-sorbose'),
 ('pLigify-SorR', 'D-glucose'),
 ('pLigify-SorR', 'D-galactose')]

In [ ]:
variant_list = [x[0] for x in variant_ligand_pairs]
variant_list

In [ ]:
# Plot each curve separately to see the possible EC50 transition
plt.rcParams["figure.figsize"] = [6, 4]

for var, linthresh in zip(variant_list, [0.001, 10, 0.1, 0.1]):
    #fig.suptitle(f'{var}', size=20)
    
    df_0 = full_summary
    df_0 = df_0[~df_0.is_bad_rep]
    df_0 = df_0[df_0.variant==var]
    x_0 = df_0.inducerConcentration
    
    ligand_list = [x[1] for x in plot_list if x[0]==var]
    
    for lig, color in zip(ligand_list, sns.color_palette()):
        fig, ax = plt.subplots(1, 1)
        df = df_0[(df_0.inducerId==lig)|(df_0.inducerId=='none')]
        if len(df)>0:
            x = df.inducerConcentration
            y = df['mean'] - non_fluorescent_mean
            yerr = df['mean_err']
            ax.errorbar(x, y/1000, yerr/1000, fmt='o', ms=ms, color=color)
            
            x_min = min(x[x>0])
            x_max = max(x)
            x_fit = [0] + list(np.logspace(np.log10(x_min/10), np.log10(x_max*2), 40))

            variant_row = variant_table[(variant_table.variant==var)&(variant_table.ligand==lig)].iloc[0]
            hill_params = [variant_row[p] for p in ['log_g0', 'log_ginf', 'log_ec50', 'n']]
            hill_params = [10**p for p in hill_params[:3]] + hill_params[3:]
            y_fit = hill_funct(x_fit, *hill_params)
            ax.plot(x_fit, y_fit/1000, color=color)
            ax.plot([], [], '-o', color=color, label=var, ms=ms)

        ax.set_xscale('symlog', linthresh=linthresh);
        ax.set_xlabel(f'[{lig}] (µmol/L)');
        ax.set_ylabel(f'Output (kMEF)');
        add_break(ax)

        ax.legend(loc='upper left', bbox_to_anchor= (1.02, 0.99), ncol=1);

In [ ]:
# Plot data with fits to check fit quality
os.chdir(notebook_dir)
os.chdir('dose_response_plots')
plt.rcParams["figure.figsize"] = [6, 4]


for var, linthresh in zip(variant_list, [0.001, 10, 0.1, 0.1]):
    fig, ax = plt.subplots(1, 1)
    #fig.suptitle(f'{var}', size=20)
    
    df_0 = full_summary
    df_0 = df_0[~df_0.is_bad_rep]
    df_0 = df_0[df_0.variant==var]
    x_0 = df_0.inducerConcentration
    
    ligand_list = [x[1] for x in plot_list if x[0]==var]
    
    for lig, color in zip(ligand_list, sns.color_palette()):
        df = df_0[(df_0.inducerId==lig)|(df_0.inducerId=='none')]
        if len(df)>0:
            x = df.inducerConcentration
            y = df['mean'] - non_fluorescent_mean
            yerr = df['mean_err']
            ax.errorbar(x, y/1000, yerr/1000, fmt='o', ms=ms, color=color)
            
            x_min = min(x[x>0])
            x_max = max(x)
            x_fit = [0] + list(np.logspace(np.log10(x_min/10), np.log10(x_max*2), 40))

            variant_row = variant_table[(variant_table.variant==var)&(variant_table.ligand==lig)].iloc[0]
            hill_params = [variant_row[p] for p in ['log_g0', 'log_ginf', 'log_ec50', 'n']]
            hill_params = [10**p for p in hill_params[:3]] + hill_params[3:]
            y_fit = hill_funct(x_fit, *hill_params)
            ax.plot(x_fit, y_fit/1000, color=color)
            ax.plot([], [], '-o', color=color, label=lig, ms=ms)

    ax.set_xscale('symlog', linthresh=linthresh);
    ax.set_xlabel(f'[ligand] (µmol/L)');
    ax.set_ylabel(f'Output (kMEF)');
    add_break(ax)
    
    ax.legend(loc='upper left', bbox_to_anchor= (1.02, 0.99), ncol=1);
    
    fig_file = f'{var}.dose_response.svg'
    fig.savefig(fig_file, bbox_inches='tight')
    
os.chdir(notebook_dir)

In [ ]:
stan_model_file = "Two-step Hill equation fit.stan"
# Load Stan model
stan_model = stan_utility.compile_model(stan_model_file)

In [ ]:
def init_stan_fit(x_data, y_data, mid_start=None):
    x_max = max(x_data)
    x_min = min(x_data)
    
    if mid_start is None:
        x_non_zero = x[x>0]
        mid_start = np.exp(np.mean(np.log(x_non_zero)))
    
    low = np.mean(y_data[x_data<=x_min])
    high = np.mean(y_data[x_data>=x_max])
    mid = np.random.normal(1, 0.2) * mid_start
    n = np.random.normal(1, 0.2) * 2
    sig = np.random.normal(1, 0.2) * 100
    
    return dict(log_low_level=np.log10(low), 
                log_mid_level=(np.log10(high) + np.log10(low))/2, 
                log_high_level=np.log10(high), 
                log_IC_50=np.log10(mid)*np.array([0.8, 1.2]), sensor_n=n,
                sigma=sig)

In [ ]:
def two_step_hill_funct(x, low, mid, high, ec50_1, ec50_2, n):
    return low + (mid-low)*( x**n )/( ec50_1**n + x**n ) + (high-mid)*( x**n )/( ec50_2**n + x**n )

In [ ]:
# Two-step Hill fit for LcLacI
gmin = 10
gmax = 150000

# Prior on sensor_n; <gamma> = alpha/beta = 2; std = sqrt(alpha)/beta = 1.2
sensor_n_alpha = 3*2
sensor_n_beta = 1.5*2

pair = ['pLigify-LcLacI', 'D-ribose']
var = pair[0]
lig = pair[1]
df = full_summary
df = df[~df.is_bad_rep]
df = df[df.variant==var]

df = df[(df.inducerId==lig)|(df.inducerId=='none')]

x = df.inducerConcentration
y = df['mean'] - non_fluorescent_mean
yerr = df['mean_err']

log_x_min = min(np.log10(x[x>0])) - 0.5
log_x_max = np.log10(max(x)) + 2

stan_data = dict(x=x, y=y, y_err=yerr, N=len(x),
                 log_g_min=np.log10(gmin), log_g_max=np.log10(gmax),
                 log_x_min=log_x_min, log_x_max=log_x_max,
                 sensor_n_alpha=sensor_n_alpha,
                 sensor_n_beta=sensor_n_beta)
stan_init = init_stan_fit(x, y)
stan_fit = stan_model.sample(data=stan_data, 
                             iter_warmup=3000,
                             iter_sampling=3000, 
                             inits=stan_init, 
                             chains=4,
                             adapt_delta=0.99, 
                             max_treedepth=15)

In [ ]:
stan_fit.summary().iloc[:13]

In [ ]:
plt.rcParams["figure.figsize"] = [6, 4]
fig, ax = plt.subplots(1, 1)

for p in ['IC_50_1', 'IC_50_2']:
    y = stan_fit.stan_variable(p)
    ax.plot(y);
ax.set_yscale('log');

In [ ]:
hill_params = ['low_level', 'mid_level', 'high_level', 'IC_50_1', 'IC_50_2', 'sensor_n']
two_step_popt = [ np.median(stan_fit.stan_variable(param)) for param in hill_params ] 

In [ ]:
# Two-step Hill fit for LcLacI
os.chdir(notebook_dir)
os.chdir('dose_response_plots')

gmin = 10
gmax = 1000000

# Prior on sensor_n; <gamma> = alpha/beta = 2; std = sqrt(alpha)/beta = 1.2
sensor_n_alpha = 3
sensor_n_beta = 1.5

plt.rcParams["figure.figsize"] = [6, 4]

pair = ['pLigify-LcLacI', 'D-ribose']
linthresh = 10
color = sns.color_palette()[0]
var = pair[0]
lig = pair[1]
df = full_summary
df = df[~df.is_bad_rep]
df = df[df.variant==var]

df = df[(df.inducerId==lig)|(df.inducerId=='none')]

fig, ax = plt.subplots(1, 1)
#fig.suptitle(f'{var}, {lig}', size=20)

x_0 = df.inducerConcentration
x_min = min(x_0[x_0>0])
x_max = max(x_0)
x_fit = [0] + list(np.logspace(np.log10(x_min/10), np.log10(x_max*2), 40))

variant_row = variant_table[(variant_table.variant==var)&(variant_table.ligand==lig)].iloc[0]
hill_params = [variant_row[p] for p in ['log_g0', 'log_ginf', 'log_ec50', 'n']]
hill_params = [10**p for p in hill_params[:3]] + hill_params[3:]
y_fit = two_step_hill_funct(x_fit, *two_step_popt)

x = df.inducerConcentration
y = df['mean'] - non_fluorescent_mean
yerr = df['mean_err']
ax.errorbar(x, y/1000, yerr/1000, fmt='o', ms=ms, color=color)

ax.plot(x_fit, y_fit/1000, color=color)

ax.set_xscale('symlog', linthresh=linthresh);
ax.set_xlabel(f'[{lig}] (µmol/L)');
ax.set_ylabel(f'Output (kMEF)');
add_break(ax)

fig_file = f'{var}_{lig}.dose_response.2-step.svg'
fig.savefig(fig_file, bbox_inches='tight')
    
os.chdir(notebook_dir)

In [ ]:
# Plot data with fits to check fit quality
os.chdir(notebook_dir)
os.chdir('dose_response_plots')
plt.rcParams["figure.figsize"] = [6, 4]


for var, linthresh in zip(['pLigify-LcLacI'], [10]):
    fig, ax = plt.subplots(1, 1)
    #fig.suptitle(f'{var}', size=20)
    
    df_0 = full_summary
    df_0 = df_0[~df_0.is_bad_rep]
    df_0 = df_0[df_0.variant==var]
    x_0 = df_0.inducerConcentration
    
    ligand_list = [x[1] for x in plot_list if x[0]==var]
    
    for lig, color in zip(ligand_list, sns.color_palette()):
        df = df_0[(df_0.inducerId==lig)|(df_0.inducerId=='none')]
        if len(df)>0:
            x = df.inducerConcentration
            y = df['mean'] - non_fluorescent_mean
            yerr = df['mean_err']
            ax.errorbar(x, y/1000, yerr/1000, fmt='o', ms=ms, color=color)
            
            x_min = min(x[x>0])
            x_max = max(x)
            x_fit = [0] + list(np.logspace(np.log10(x_min/10), np.log10(x_max*2), 40))
            
            if lig == 'D-ribose':
                y_fit = two_step_hill_funct(x_fit, *two_step_popt)
            else:
                variant_row = variant_table[(variant_table.variant==var)&(variant_table.ligand==lig)].iloc[0]
                hill_params = [variant_row[p] for p in ['log_g0', 'log_ginf', 'log_ec50', 'n']]
                hill_params = [10**p for p in hill_params[:3]] + hill_params[3:]
                y_fit = hill_funct(x_fit, *hill_params)
            ax.plot(x_fit, y_fit/1000, color=color)
            ax.plot([], [], '-o', color=color, label=lig, ms=ms)

    ax.set_xscale('symlog', linthresh=linthresh);
    ax.set_xlabel(f'[ligand] (µmol/L)');
    ax.set_ylabel(f'Output (kMEF)');
    add_break(ax)
    
    ax.legend(loc='upper left', bbox_to_anchor= (1.02, 0.99), ncol=1);
    
    fig_file = f'{var}.dose_response.2-step.svg'
    fig.savefig(fig_file, bbox_inches='tight')
    
os.chdir(notebook_dir)

In [ ]:
for p in ['IC_50_1', 'IC_50_2', 'low_level', 'high_level', 'sensor_n']:
    print(f'{np.mean(stan_fit.stan_variable(p)):.3f} +- {np.std(stan_fit.stan_variable(p)):.3f}')

In [ ]:
g_ratio = stan_fit.stan_variable('high_level')/stan_fit.stan_variable('low_level')
print(f'{np.mean(g_ratio):.3f} +- {np.std(g_ratio):.3f}')

In [ ]:
g_ratio = stan_fit.stan_variable('max_level')/stan_fit.stan_variable('low_level')
print(np.quantile(g_ratio, 0.05))

In [ ]:
g_max = stan_fit.stan_variable('max_level')
print(np.quantile(g_max, 0.05))

In [ ]:
# Lower limit for EC50_2
np.quantile(stan_fit.stan_variable('IC_50_2'), 0.05)